In [1]:
import pandas as pd
import numpy as np
import os
import re
data_base = r"C:\Users\Administrator\Downloads\bai_tap_nhom_case11"
data_raw = os.path.join(data_base,"data_raw")
data_clean = os.path.join(data_base,"data_clean")

os.makedirs(data_clean, exist_ok=True)

df_customer = pd.read_csv(os.path.join(data_raw,'customer_profile.csv'))
df_consumption = pd.read_csv(os.path.join(data_raw,'consumption_log.csv'))
df_tariff = pd.read_csv(os.path.join(data_raw,'price_tariff.csv'))

print(f"Đã tải thành công 3 bảng dữ liệu: Khách hàng ({len(df_customer)} dòng), Tiêu thụ ({len(df_consumption)} dòng), Bảng giá ({len(df_tariff)} dòng).")


PermissionError: [WinError 5] Access is denied: 'C:\\Users\\Administrator'

**Chuẩn hóa thông tin**


In [ ]:
# customer_id: Chuyển HOA và xóa khoảng trắng
df_customer['customer_id'] = df_customer['customer_id'].astype(str).str.upper().str.strip()

# house_type: Chuẩn hóa tên loại nhà
df_customer['house_type'] = (df_customer['house_type']
                             .astype(str).str.strip().str.lower()
                             .replace({'chung cu': 'Chung Cư', 'tap the': 'Tập Thể',
                                       'biet thu': 'Biệt Thự', 'nha pho': 'Nhà Phố'})
                             .str.title()
                            )

# region: Chuẩn hóa tên khu vực/tỉnh thành
df_customer['region'] = (df_customer['region']
                         .astype(str).str.strip().str.lower()
                         .replace({'ha noi': 'Hà Nội', 'tp.hcm': 'TP. Hồ Chí Minh', 
                                   'tp hcm': 'TP. Hồ Chí Minh', 'bac ninh': 'Bắc Ninh',
                                   'bac  ninh': 'Bắc Ninh', 'da nang': 'Đà Nẵng'})
                         .str.title()
                        )
df_customer_clean = df_customer
print("\n[customer_profile - 5 dòng đã làm sạch]:")
print(df_customer_clean.head())                        


[customer_profile - 5 dòng đã làm sạch]:
  customer_id house_type           region
0       KH100   Chung Cư           Hà Nội
1       KH101   Biệt Thự           Hà Nội
2       KH102   Chung Cư           Hà Nội
3       KH103    Tập Thể  Tp. Hồ Chí Minh
4       KH104    Tập Thể         Bắc Ninh


**Xử lí và tính toán**

In [ ]:
# customer_id & meter_id: Chuyển HOA và xóa khoảng trắng
df_consumption['customer_id'] = df_consumption['customer_id'].astype(str).str.upper().str.strip()
df_consumption['meter_id'] = df_consumption['meter_id'].astype(str).str.upper().str.strip()

# reading_date: Chuyển về định dạng datetime thống nhất (xử lý cả DD/MM/YYYY và YYYY-MM-DD)
df_consumption['reading_date'] = pd.to_datetime(df_consumption['reading_date'], errors='coerce', dayfirst=True)

# consumption_kwh: Loại bỏ text, xử lý giá trị âm/NaN
df_consumption['consumption_kwh'] = (df_consumption['consumption_kwh']
                                     .astype(str)
                                     .str.replace(r'[^\d\.-]', '', regex=True) # Chỉ giữ số, dấu chấm, dấu trừ
                                     .str.strip()
                                    )
df_consumption['consumption_kwh'] = pd.to_numeric(df_consumption['consumption_kwh'], errors='coerce')
# Xử lý giá trị âm bất thường (thay bằng NaN)
df_consumption.loc[df_consumption['consumption_kwh'] < 0, 'consumption_kwh'] = np.nan
df_consumption_clean = df_consumption.dropna(subset=['reading_date', 'consumption_kwh']).reset_index(drop=True)
print("\n[consumption_log - 5 dòng đã làm sạch]:")
print(df_consumption_clean.head())

# Loại bỏ text và chuyển đổi sang dạng số
df_tariff['kwh_min'] = df_tariff['kwh_min'].astype(str).str.replace(r'[^\d]', '', regex=True).str.strip()
df_tariff['kwh_max'] = df_tariff['kwh_max'].astype(str).str.replace(r'[^\d]', '', regex=True).str.strip()
df_tariff['price_per_kwh'] = df_tariff['price_per_kwh'].astype(str).str.replace(r'[^\d]', '', regex=True).str.strip()
df_tariff['kwh_min'] = pd.to_numeric(df_tariff['kwh_min'], errors='coerce')
df_tariff['kwh_max'] = pd.to_numeric(df_tariff['kwh_max'], errors='coerce')
df_tariff['price_per_kwh'] = pd.to_numeric(df_tariff['price_per_kwh'], errors='coerce')

# Xử lý bậc thang cuối có kwh_max bị trống: đặt là vô hạn (np.inf)
df_tariff.loc[df_tariff['tariff_level'] == 'B5', 'kwh_max'] = np.inf
df_tariff_clean = df_tariff.dropna(subset=['kwh_min', 'price_per_kwh']).reset_index(drop=True)
print("\n[price_tariff - Bảng giá đã làm sạch]:")
print(df_tariff_clean)


[consumption_log - 5 dòng đã làm sạch]:
  customer_id meter_id reading_date  consumption_kwh
0       KH136    CT016   2024-01-27             50.0
1       KH126    CT999   2024-03-06            150.0
2       KH103    CT007   2024-02-29             75.0
3       KH113    CT009   2024-03-23            100.0
4       KH138    CT008   2024-01-15             50.0

[price_tariff - Bảng giá đã làm sạch]:
  tariff_level  kwh_min  kwh_max  price_per_kwh
0           B1        0     50.0           1800
1           B2       51    100.0           2000
2           B3      101    200.0           2500
3           B4      201    300.0           3000
4           B5      301      inf           3500


**Xuất dữ liệu**

In [ ]:
data_clean = os.path.join(data_base, "data_clean")
os.makedirs(data_clean, exist_ok=True)

print("\n--- Xuất file sạch (Clean Files) ---")

df_customer_clean.to_csv(os.path.join(data_clean, "customer_profile_clean.csv"),index=False)

df_consumption_clean.to_csv(os.path.join(data_clean, "consumption_log_clean.csv"),index=False)

df_tariff_clean.to_csv(os.path.join(data_clean, "price_tariff_clean.csv"),index=False)

print("\n Hoàn thành việc làm sạch và xuất file.")



--- Xuất file sạch (Clean Files) ---

 Hoàn thành việc làm sạch và xuất file.


**Hoan thien task 1**
